https://stackoverflow.com/questions/47826730/how-to-save-resized-images-using-imagedatagenerator-and-flow-from-directory-in-k

In [77]:
import os

In [78]:
os.chdir(r"C:\Users\bishw\Downloads\Compressed\dataset")

In [79]:
os.listdir()

['sample.npz', 'single_prediction', 'test_set', 'training_set']

In [80]:
train = r"C:\Users\bishw\Downloads\Compressed\dataset\training_set"

In [81]:
os.listdir(train)

['cats', 'dogs']

In [82]:
import matplotlib.pyplot as plt
from matplotlib.image import imread

In [83]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

In [84]:
std_img_size = (64,64,3)

In [85]:
img_gen = ImageDataGenerator(rescale = 1./255, rotation_range = 20, width_shift_range = 0.1, height_shift_range = 0.1, shear_range = 0.1, zoom_range = 0.1, horizontal_flip = True, fill_mode = 'nearest')

In [86]:
import numpy as np

In [87]:

for i in os.listdir(train):
    if i == 'cats':
        for j in os.listdir(train+'\\'+i):
            image = img_to_array(load_img(train+'\\'+i+'\\'+j, target_size = std_img_size))
            image = np.expand_dims(image, axis = 0)
            img_gen.fit(image)
            for x, val in zip(img_gen.flow(image, batch_size = 32, save_to_dir=r"C:\Users\bishw\Downloads\Compressed\dataset\training_set\cats", save_prefix = 'aug_'), range(30)):
                pass
    else:
        for j in os.listdir(train+'\\'+i):
            image = img_to_array(load_img(train+'\\'+i+'\\'+j, target_size = std_img_size))
            image = np.expand_dims(image, axis = 0)
            img_gen.fit(image)
            for x, val in zip(img_gen.flow(image, batch_size = 32, save_to_dir=r"C:\Users\bishw\Downloads\Compressed\dataset\training_set\dogs", save_prefix = 'aug_'), range(30)):
                pass
        
        

In [25]:
import random2

def make_val_set(PATH, p):
    PATH = PATH if PATH[-1] == '/' else PATH+'/'
    list_folders = os.listdir(f"{PATH}") 
    for i in list_folders: 
        os.makedirs(f"{PATH}valid/{i}", exist_ok=True)
        list_of_files = os.listdir(f"{PATH}/{i}")
        random2.shuffle(list_of_files)
        n_idxs = int(len(list_of_files)*p)
        selected_files = [list_of_files[n] for n in range(n_idxs)]
        for file in selected_files:
            os.rename(f"{PATH}/{i}/{file}", f"{PATH}valid/{i}/{file}")

make_val_set(train, 0.2)

In [27]:
import shutil

In [29]:
shutil.move(r"D:\self_identification_dataset\train\valid", r"D:\self_identification_dataset")

'D:\\self_identification_dataset\\valid'

In [88]:
train_tensor = []
train_labels = []
for i in os.listdir(train):
    if i == 'dogs':
        for j in os.listdir(train+'\\'+i):
            image = img_to_array(load_img(train+'\\'+i+'\\'+j, target_size = std_img_size))
            train_tensor.append(image)
            train_labels.append(1)
    else:
        for j in os.listdir(train+'\\'+i):
            image = img_to_array(load_img(train+'\\'+i+'\\'+j, target_size = std_img_size))
            train_tensor.append(image)
            train_labels.append(0)
        

In [89]:
len(train_tensor)

28000

In [90]:
valid = r"C:\Users\bishw\Downloads\Compressed\dataset\test_set"

In [91]:
test_tensor = []
test_labels = []
for i in os.listdir(valid):
    if i == 'dogs':
        for j in os.listdir(valid+'\\'+i):
            image = img_to_array(load_img(valid+'\\'+i+'\\'+j, target_size = std_img_size))
            test_tensor.append(image)
            test_labels.append(1)
    else:
        for j in os.listdir(valid+'\\'+i):
            image = img_to_array(load_img(valid+'\\'+i+'\\'+j, target_size = std_img_size))
            test_tensor.append(image)
            test_labels.append(0)
        

In [92]:
len(test_tensor)

2000

In [93]:
# Using numpy's savez function to store our loaded data as NPZ files
np.savez('dog_vs_cat_train_tensor.npz', np.array(train_tensor))
np.savez('dog_vs_cat_train_labels.npz', np.array(train_labels))
np.savez('dog_vs_cat_test_tensor.npz', np.array(test_tensor))
np.savez('dog_vs_cat_test_labels.npz', np.array(test_labels))

In [94]:

# Loader Function
import numpy as np

def load_data_training_and_test(datasetname):
    
    npzfile = np.load(datasetname + "_train_tensor.npz")
    train = npzfile['arr_0']
    
    npzfile = np.load(datasetname + "_train_labels.npz")
    train_labels = npzfile['arr_0']
    
    npzfile = np.load(datasetname + "_test_tensor.npz")
    test = npzfile['arr_0']
    
    npzfile = np.load(datasetname + "_test_labels.npz")
    test_labels = npzfile['arr_0']

    return (train, train_labels), (test, test_labels)

In [95]:
(X_train, y_train), (X_test, y_test) = load_data_training_and_test('dog_vs_cat')

In [96]:
X_train.shape

(28000, 64, 64, 3)

In [97]:
X_train = X_train.reshape(len(X_train),X_train.shape[1],X_train.shape[2],X_train.shape[3])
X_test = X_test.reshape(len(X_test),X_test.shape[1],X_test.shape[2],X_test.shape[3])

In [106]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, MaxPooling2D, Conv2D, Dense, Flatten

In [107]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3,3),input_shape=std_img_size, activation='relu'))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=std_img_size, activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding = 'same'))

model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(128, activation = 'relu'))

model.add(Dropout(0.5))

model.add(Dense(1,activation = 'sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [108]:
from tensorflow.keras.callbacks import EarlyStopping

In [109]:
early_stop = EarlyStopping(monitor = 'accuracy', patience = 3)

In [111]:
model.fit(X_train, y_train, batch_size = 50, epochs = 5, callbacks = [early_stop], validation_data = (X_test, y_test))

Epoch 1/5


186/560 [========>.....................] - ETA: 0s - loss: 0.6989 - accuracy: 0.44 - ETA: 3:28 - loss: 0.6843 - accuracy: 0.53 - ETA: 5:18 - loss: 0.6779 - accuracy: 0.53 - ETA: 4:58 - loss: 0.6756 - accuracy: 0.55 - ETA: 4:43 - loss: 0.6762 - accuracy: 0.56 - ETA: 4:37 - loss: 0.6745 - accuracy: 0.56 - ETA: 4:38 - loss: 0.6725 - accuracy: 0.55 - ETA: 4:40 - loss: 0.6712 - accuracy: 0.55 - ETA: 4:35 - loss: 0.6727 - accuracy: 0.55 - ETA: 4:31 - loss: 0.6726 - accuracy: 0.55 - ETA: 4:26 - loss: 0.6729 - accuracy: 0.55 - ETA: 4:23 - loss: 0.6760 - accuracy: 0.54 - ETA: 4:20 - loss: 0.6777 - accuracy: 0.53 - ETA: 4:17 - loss: 0.6784 - accuracy: 0.53 - ETA: 4:14 - loss: 0.6771 - accuracy: 0.53 - ETA: 4:15 - loss: 0.6794 - accuracy: 0.54 - ETA: 4:12 - loss: 0.6821 - accuracy: 0.54 - ETA: 4:11 - loss: 0.6804 - accuracy: 0.54 - ETA: 4:08 - loss: 0.6808 - accuracy: 0.53 - ETA: 4:07 - loss: 0.6794 - accuracy: 0.54 - ETA: 4:07 - loss: 0.6784 - accuracy: 0.54 - ETA: 4:07 - loss: 0.6780 - accuracy

372/560 [==================>...........] - ETA: 2:41 - loss: 0.6841 - accuracy: 0.54 - ETA: 2:40 - loss: 0.6841 - accuracy: 0.54 - ETA: 2:40 - loss: 0.6840 - accuracy: 0.54 - ETA: 2:39 - loss: 0.6840 - accuracy: 0.54 - ETA: 2:39 - loss: 0.6840 - accuracy: 0.54 - ETA: 2:38 - loss: 0.6839 - accuracy: 0.54 - ETA: 2:38 - loss: 0.6839 - accuracy: 0.54 - ETA: 2:38 - loss: 0.6837 - accuracy: 0.54 - ETA: 2:37 - loss: 0.6837 - accuracy: 0.54 - ETA: 2:37 - loss: 0.6837 - accuracy: 0.54 - ETA: 2:36 - loss: 0.6837 - accuracy: 0.54 - ETA: 2:36 - loss: 0.6838 - accuracy: 0.54 - ETA: 2:35 - loss: 0.6836 - accuracy: 0.54 - ETA: 2:35 - loss: 0.6837 - accuracy: 0.54 - ETA: 2:34 - loss: 0.6835 - accuracy: 0.54 - ETA: 2:34 - loss: 0.6835 - accuracy: 0.54 - ETA: 2:33 - loss: 0.6835 - accuracy: 0.54 - ETA: 2:33 - loss: 0.6836 - accuracy: 0.54 - ETA: 2:32 - loss: 0.6837 - accuracy: 0.54 - ETA: 2:32 - loss: 0.6835 - accuracy: 0.54 - ETA: 2:31 - loss: 0.6835 - accuracy: 0.54 - ETA: 2:31 - loss: 0.6833 - accura

559/560 [============================>.] - ETA: 1:18 - loss: 0.6858 - accuracy: 0.54 - ETA: 1:18 - loss: 0.6857 - accuracy: 0.54 - ETA: 1:17 - loss: 0.6857 - accuracy: 0.54 - ETA: 1:17 - loss: 0.6857 - accuracy: 0.54 - ETA: 1:16 - loss: 0.6857 - accuracy: 0.54 - ETA: 1:16 - loss: 0.6858 - accuracy: 0.54 - ETA: 1:16 - loss: 0.6857 - accuracy: 0.54 - ETA: 1:15 - loss: 0.6857 - accuracy: 0.54 - ETA: 1:15 - loss: 0.6856 - accuracy: 0.54 - ETA: 1:14 - loss: 0.6857 - accuracy: 0.54 - ETA: 1:14 - loss: 0.6857 - accuracy: 0.54 - ETA: 1:13 - loss: 0.6856 - accuracy: 0.54 - ETA: 1:13 - loss: 0.6857 - accuracy: 0.54 - ETA: 1:13 - loss: 0.6857 - accuracy: 0.54 - ETA: 1:12 - loss: 0.6857 - accuracy: 0.54 - ETA: 1:12 - loss: 0.6857 - accuracy: 0.54 - ETA: 1:11 - loss: 0.6861 - accuracy: 0.54 - ETA: 1:11 - loss: 0.6860 - accuracy: 0.54 - ETA: 1:11 - loss: 0.6859 - accuracy: 0.54 - ETA: 1:10 - loss: 0.6860 - accuracy: 0.54 - ETA: 1:10 - loss: 0.6860 - accuracy: 0.54 - ETA: 1:09 - loss: 0.6860 - accura

560/560 [==============================] - ETA: 0s - loss: 0.6843 - accuracy: 0.55 - 239s 426ms/step - loss: 0.6843 - accuracy: 0.5551 - val_loss: 0.6845 - val_accuracy: 0.5655
Epoch 2/5


...

560/560 [==============================] - ETA: 0s - loss: 0.6135 - accuracy: 0.66 - 207s 370ms/step - loss: 0.6135 - accuracy: 0.6611 - val_loss: 0.6676 - val_accuracy: 0.6500
Epoch 5/5


186/560 [========>.....................] - ETA: 0s - loss: 0.6400 - accuracy: 0.62 - ETA: 2:08 - loss: 0.6083 - accuracy: 0.68 - ETA: 2:56 - loss: 0.5951 - accuracy: 0.64 - ETA: 3:27 - loss: 0.5766 - accuracy: 0.66 - ETA: 3:42 - loss: 0.5635 - accuracy: 0.66 - ETA: 3:45 - loss: 0.5505 - accuracy: 0.68 - ETA: 3:45 - loss: 0.5680 - accuracy: 0.68 - ETA: 3:43 - loss: 0.5657 - accuracy: 0.68 - ETA: 3:48 - loss: 0.5610 - accuracy: 0.69 - ETA: 3:52 - loss: 0.5572 - accuracy: 0.69 - ETA: 3:54 - loss: 0.5585 - accuracy: 0.68 - ETA: 3:56 - loss: 0.5592 - accuracy: 0.67 - ETA: 3:55 - loss: 0.5576 - accuracy: 0.68 - ETA: 3:52 - loss: 0.5600 - accuracy: 0.68 - ETA: 3:51 - loss: 0.5613 - accuracy: 0.68 - ETA: 3:50 - loss: 0.5560 - accuracy: 0.68 - ETA: 3:52 - loss: 0.5624 - accuracy: 0.68 - ETA: 3:51 - loss: 0.5656 - accuracy: 0.68 - ETA: 3:49 - loss: 0.5620 - accuracy: 0.68 - ETA: 3:47 - loss: 0.5611 - accuracy: 0.68 - ETA: 3:45 - loss: 0.5623 - accuracy: 0.68 - ETA: 3:44 - loss: 0.5611 - accuracy

372/560 [==================>...........] - ETA: 2:31 - loss: 0.5634 - accuracy: 0.70 - ETA: 2:31 - loss: 0.5635 - accuracy: 0.70 - ETA: 2:31 - loss: 0.5634 - accuracy: 0.70 - ETA: 2:30 - loss: 0.5634 - accuracy: 0.70 - ETA: 2:30 - loss: 0.5635 - accuracy: 0.70 - ETA: 2:30 - loss: 0.5632 - accuracy: 0.70 - ETA: 2:29 - loss: 0.5631 - accuracy: 0.70 - ETA: 2:29 - loss: 0.5633 - accuracy: 0.70 - ETA: 2:29 - loss: 0.5631 - accuracy: 0.70 - ETA: 2:29 - loss: 0.5634 - accuracy: 0.70 - ETA: 2:28 - loss: 0.5630 - accuracy: 0.70 - ETA: 2:28 - loss: 0.5630 - accuracy: 0.70 - ETA: 2:27 - loss: 0.5628 - accuracy: 0.70 - ETA: 2:27 - loss: 0.5631 - accuracy: 0.70 - ETA: 2:26 - loss: 0.5628 - accuracy: 0.70 - ETA: 2:26 - loss: 0.5631 - accuracy: 0.70 - ETA: 2:26 - loss: 0.5629 - accuracy: 0.70 - ETA: 2:25 - loss: 0.5627 - accuracy: 0.70 - ETA: 2:25 - loss: 0.5625 - accuracy: 0.70 - ETA: 2:24 - loss: 0.5625 - accuracy: 0.70 - ETA: 2:24 - loss: 0.5623 - accuracy: 0.70 - ETA: 2:24 - loss: 0.5625 - accura

559/560 [============================>.] - ETA: 1:17 - loss: 0.5670 - accuracy: 0.70 - ETA: 1:17 - loss: 0.5673 - accuracy: 0.70 - ETA: 1:16 - loss: 0.5675 - accuracy: 0.70 - ETA: 1:16 - loss: 0.5675 - accuracy: 0.70 - ETA: 1:16 - loss: 0.5675 - accuracy: 0.70 - ETA: 1:15 - loss: 0.5676 - accuracy: 0.70 - ETA: 1:15 - loss: 0.5673 - accuracy: 0.70 - ETA: 1:14 - loss: 0.5673 - accuracy: 0.70 - ETA: 1:14 - loss: 0.5676 - accuracy: 0.70 - ETA: 1:13 - loss: 0.5676 - accuracy: 0.70 - ETA: 1:13 - loss: 0.5674 - accuracy: 0.70 - ETA: 1:13 - loss: 0.5674 - accuracy: 0.70 - ETA: 1:12 - loss: 0.5672 - accuracy: 0.70 - ETA: 1:12 - loss: 0.5671 - accuracy: 0.70 - ETA: 1:11 - loss: 0.5668 - accuracy: 0.70 - ETA: 1:11 - loss: 0.5667 - accuracy: 0.70 - ETA: 1:11 - loss: 0.5668 - accuracy: 0.70 - ETA: 1:10 - loss: 0.5671 - accuracy: 0.70 - ETA: 1:10 - loss: 0.5674 - accuracy: 0.70 - ETA: 1:09 - loss: 0.5673 - accuracy: 0.70 - ETA: 1:09 - loss: 0.5671 - accuracy: 0.70 - ETA: 1:08 - loss: 0.5674 - accura

560/560 [==============================] - ETA: 0s - loss: 0.5745 - accuracy: 0.69 - 244s 435ms/step - loss: 0.5745 - accuracy: 0.6990 - val_loss: 0.6581 - val_accuracy: 0.6695


In [98]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch


def build_model(hp):  
    model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=std_img_size
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='binary_crossentropy',
              metrics=['accuracy'])
    return model

In [99]:
from kerastuner import HyperParameters, RandomSearch


In [100]:
tuner = RandomSearch(build_model, objective = 'val_accuracy', max_trials = 5, directory = 'output', project_name = 'self_iden')


tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))